### Reddit Data Scrapper Notebook

This notebook contains the code to scrap the data of Reddit India using python PRAW(Python Reddit API Wrapper) Library. I have scrapped the top 150 posts of Reddit India. Total 12 different flairs posts has been scrapped. For each of the posts Title, Score, URL, Body, ID, Authors etc has been scrapped. Also the top 20 comments of each posts has been scrapped

In [1]:
import pymongo
from pymongo import MongoClient
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords


client = MongoClient('mongodb://localhost:27017/')
db = client["reddit"]

In [2]:
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='flare_detector', \
                     username='', \
                     password='')

In [3]:
subreddit = reddit.subreddit('india')

In [4]:
#List of all the flairs. These will be the keys in classification.
flairs = ["AskIndia", "Unverified", "Non-Political", 
          "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy",
          "Sports", "Food", "[R]eddiquette"]


for flair in flairs:

    #The posts' data is collected by searching by the flair name in the list. Top 150 posts are recorded and stored for analysis.

    relevant_subreddits = subreddit.search(f"flair_name:{flair}",limit=150)

    for submission in relevant_subreddits:
        posts = {
        "title":str(submission.title),
        "score":str(submission.score),
        "id":str(submission.id),
        "url":str(submission.url),
        "comms_num":str(submission.num_comments),
        "created":str(submission.created),
        "body":str(submission.selftext),
        "author":str(submission.author),
        "flair":str(flair),	
        }

#  Only top twenty comments and their authors are considered for the data. 
        submission.comments.replace_more(limit=None)
        comment = ''
        authors = ''
        count = 0
        for top_level_comment in submission.comments:
            comment = comment + ' ' + top_level_comment.body
            authors = authors + ' ' + str(top_level_comment.author)
            count+=1     
            if(count > 20):
                break

        posts["comment"] = str(comment)
        posts["authors"] = str(authors)
        result = db.posts.insert_one(posts)